<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!unzip data.zip

Archive:  data.zip
  inflating: quora.csv               
  inflating: __MACOSX/._quora.csv    


In [8]:
!pip install pandas scikit-learn matplotlib

In [0]:
import tensorflow as tf

In [0]:
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from tensorflow.keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
quora = pd.read_csv('quora.csv')

In [0]:
vocab = Counter()

for sym in quora.question_text:
    sym = sym.lower()
    vocab.update(sym)

In [14]:
len(vocab)

1949

In [15]:
vocab

Counter({'h': 3851337,
         'o': 6026222,
         'w': 2121186,
         ' ': 15416812,
         'd': 2683873,
         'i': 5826579,
         'q': 88696,
         'u': 2073775,
         'e': 8363288,
         'b': 1133229,
         'c': 2396335,
         'n': 5009215,
         'a': 6388387,
         't': 6225943,
         'l': 2828768,
         's': 4721050,
         'r': 4375590,
         'p': 1574574,
         'v': 744930,
         '1': 101658,
         '9': 20498,
         '6': 20285,
         '0': 107880,
         '?': 1381192,
         'y': 1592600,
         'g': 1523009,
         ',': 235096,
         'f': 1541895,
         'm': 2002799,
         'k': 628955,
         'j': 147382,
         'z': 64312,
         '.': 134056,
         '/': 42137,
         'x': 164725,
         '3': 32458,
         "'": 163763,
         '2': 77956,
         '8': 24711,
         '-': 67094,
         '(': 56618,
         ')': 56594,
         '7': 25697,
         '"': 73272,
         '5': 35019,
 

In [0]:
filtered_vocab = set()

for sym in vocab:
    if vocab[sym] > 5:
        filtered_vocab.add(sym)

In [17]:
len(filtered_vocab)

413

In [0]:
sym2id = {'UNK':1, 'PAD':0}

for sym in filtered_vocab:
    sym2id[sym] = len(sym2id)

In [19]:
sym2id

{'\n': 139,
 ' ': 170,
 '!': 73,
 '"': 211,
 '#': 62,
 '$': 406,
 '%': 158,
 '&': 68,
 "'": 109,
 '(': 235,
 ')': 245,
 '*': 259,
 '+': 65,
 ',': 269,
 '-': 157,
 '.': 129,
 '/': 91,
 '0': 166,
 '1': 117,
 '2': 388,
 '3': 349,
 '4': 182,
 '5': 266,
 '6': 63,
 '7': 175,
 '8': 191,
 '9': 356,
 ':': 309,
 ';': 110,
 '<': 97,
 '=': 325,
 '>': 8,
 '?': 241,
 '@': 229,
 'PAD': 0,
 'UNK': 1,
 '[': 278,
 '\\': 348,
 ']': 180,
 '^': 340,
 '_': 289,
 '`': 181,
 'a': 57,
 'b': 50,
 'c': 324,
 'd': 294,
 'e': 306,
 'f': 78,
 'g': 122,
 'h': 227,
 'i': 187,
 'j': 374,
 'k': 299,
 'l': 305,
 'm': 255,
 'n': 126,
 'o': 82,
 'p': 293,
 'q': 226,
 'r': 33,
 's': 344,
 't': 367,
 'u': 264,
 'v': 64,
 'w': 12,
 'x': 263,
 'y': 359,
 'z': 298,
 '{': 346,
 '|': 199,
 '}': 19,
 '~': 140,
 '\xa0': 77,
 '¡': 133,
 '£': 354,
 '¨': 236,
 '«': 216,
 '¬': 41,
 '\xad': 256,
 '®': 92,
 '¯': 287,
 '°': 343,
 '±': 363,
 '²': 369,
 '³': 267,
 '´': 177,
 '·': 376,
 '¹': 192,
 'º': 239,
 '»': 203,
 '½': 205,
 '¿': 160,


In [0]:
id2sym = {i:sym for sym, i in sym2id.items()}

In [21]:
id2sym

{0: 'PAD',
 1: 'UNK',
 2: 'న',
 3: 'く',
 4: '不',
 5: '–',
 6: '）',
 7: 'ย',
 8: '>',
 9: '人',
 10: '谢',
 11: 'ι',
 12: 'w',
 13: '\u202a',
 14: 'ج',
 15: '÷',
 16: '的',
 17: 'ć',
 18: 'ş',
 19: '}',
 20: '罗',
 21: 'đ',
 22: 'া',
 23: '≠',
 24: 'ت',
 25: 'ா',
 26: '℅',
 27: '日',
 28: 'ल',
 29: 'に',
 30: '子',
 31: '是',
 32: 'प',
 33: 'r',
 34: '—',
 35: 'θ',
 36: '⊆',
 37: 'ै',
 38: 'ь',
 39: 'न',
 40: 'ะ',
 41: '¬',
 42: 'ح',
 43: '️',
 44: 'ș',
 45: '民',
 46: '卐',
 47: 'ς',
 48: 'ï',
 49: 'ら',
 50: 'b',
 51: 'い',
 52: 'ு',
 53: '∅',
 54: 'し',
 55: 'ត',
 56: '\u200e',
 57: 'a',
 58: 'த',
 59: '−',
 60: 'ख',
 61: 'ب',
 62: '#',
 63: '6',
 64: 'v',
 65: '+',
 66: 'ज',
 67: 'ū',
 68: '&',
 69: '̂',
 70: '∑',
 71: 'ā',
 72: 'ग',
 73: '!',
 74: '͊',
 75: 'ç',
 76: 'ř',
 77: '\xa0',
 78: 'f',
 79: '∫',
 80: 'ण',
 81: 'ु',
 82: 'o',
 83: 'є',
 84: '≡',
 85: '″',
 86: 'î',
 87: 'о',
 88: 'ह',
 89: 'μ',
 90: '中',
 91: '/',
 92: '®',
 93: 'е',
 94: 'ì',
 95: 'ন',
 96: 'ف',
 97: '<',
 98: '国',
 99

In [0]:
X = []

for text in quora.question_text:
    text = text.lower()
    ids = [sym2id.get(sym, 1) for sym in text]
    X.append(ids)

In [23]:
X

[[227,
  82,
  12,
  170,
  294,
  187,
  294,
  170,
  226,
  264,
  306,
  50,
  306,
  324,
  170,
  126,
  57,
  367,
  187,
  82,
  126,
  57,
  305,
  187,
  344,
  367,
  344,
  170,
  344,
  306,
  306,
  170,
  367,
  227,
  306,
  187,
  33,
  170,
  293,
  33,
  82,
  64,
  187,
  126,
  324,
  306,
  170,
  57,
  344,
  170,
  57,
  170,
  126,
  57,
  367,
  187,
  82,
  126,
  170,
  187,
  126,
  170,
  367,
  227,
  306,
  170,
  117,
  356,
  63,
  166,
  344,
  241],
 [294,
  82,
  170,
  359,
  82,
  264,
  170,
  227,
  57,
  64,
  306,
  170,
  57,
  126,
  170,
  57,
  294,
  82,
  293,
  367,
  306,
  294,
  170,
  294,
  82,
  122,
  269,
  170,
  227,
  82,
  12,
  170,
  12,
  82,
  264,
  305,
  294,
  170,
  359,
  82,
  264,
  170,
  306,
  126,
  324,
  82,
  264,
  33,
  57,
  122,
  306,
  170,
  293,
  306,
  82,
  293,
  305,
  306,
  170,
  367,
  82,
  170,
  57,
  294,
  82,
  293,
  367,
  170,
  57,
  126,
  294,
  170,
  126,
  82,
  367,
  170,


In [0]:
MAX_LEN = max(len(x) for x in X)

In [0]:
MEAN_LEN = np.median([len(x) for x in X])

In [26]:
MAX_LEN, MEAN_LEN

(1017, 60.0)

In [0]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)

In [28]:
X.shape

(1306122, 1017)

In [0]:
y = quora.target.values

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)

In [0]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.weights', 
                                                monitor='val_f1',
                                                verbose=1,
                                                save_weights_only=True,
                                                save_best_only=True,
                                                mode='max',
                                                save_freq='epoch'
                                               )

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_f1', 
                                              min_delta=0.01,
                                              patience=3,
                                              verbose=1, 
                                              mode='max',
                                              )

In [0]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))

embeddings = tf.keras.layers.Embedding(input_dim=len(sym2id), output_dim=100)(inputs)
drop1 = tf.keras.layers.Dropout(0.3)(embeddings)
conv1 = tf.keras.layers.Conv1D(kernel_size=3, filters=128, strides=1, 
                                               kernel_regularizer='l2',
                                              activation='relu')(embeddings)
conv2 = tf.keras.layers.Conv1D(kernel_size=5, filters=128, strides=2, 
                                               kernel_regularizer='l2',
                                              activation='relu')(conv1)

drop2 = tf.keras.layers.Dropout(0.5)(conv2)

flatten = tf.keras.layers.Flatten()(drop2)
dense = tf.keras.layers.Dense(50, activation='relu')(flatten)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=[f1])

In [0]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2000,
          epochs=10,
          callbacks=[checkpoint, early_stop])

Epoch 1/10
621/621 [==============================] - ETA: 0s - loss: 0.3321 - f1: 1.7705e-04 
Epoch 00001: val_f1 improved from -inf to 0.00000, saving model to model.weights
621/621 [==============================] - 6576s 11s/step - loss: 0.3321 - f1: 1.7705e-04 - val_loss: 0.1953 - val_f1: 0.0000e+00
Epoch 2/10
621/621 [==============================] - ETA: 0s - loss: 0.1869 - f1: 0.0841 
Epoch 00002: val_f1 improved from 0.00000 to 0.10958, saving model to model.weights
621/621 [==============================] - 6572s 11s/step - loss: 0.1869 - f1: 0.0841 - val_loss: 0.1803 - val_f1: 0.1096
Epoch 3/10
621/621 [==============================] - ETA: 0s - loss: 0.1767 - f1: 0.2319 
Epoch 00003: val_f1 improved from 0.10958 to 0.20701, saving model to model.weights
621/621 [==============================] - 6546s 11s/step - loss: 0.1767 - f1: 0.2319 - val_loss: 0.1726 - val_f1: 0.2070
Epoch 4/10
621/621 [==============================] - ETA: 0s - loss: 0.1697 - f1: 0.3003 
Epoch 000